In [34]:
import os
if "x_perceiver" not in os.listdir():
    os.chdir("/home/kh701/pycharm/healnet/")
import torch
from torch import nn
import multiprocessing
from typing import *
import torchvision
import numpy as np
import torchvision.transforms as transforms
import einops
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from healnet.models.explainer import Explainer
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

from healnet.utils import Config, flatten_config
from healnet.etl import TCGADataset
from healnet.models.healnet import Attention, PreNorm
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

    
%reload_ext autoreload
%autoreload 2

## Import data

In [35]:
# get dataloaders
config = Config("config/main_gpu.yml").read()
config = flatten_config(config) # TODO - refactor to other 

blca = TCGADataset(
    dataset="blca", 
    config=config, 
    level=2, 
    sources=["omic"]
)

brca = TCGADataset(
    dataset="brca", 
    config=config, 
    level=2, 
    sources=["omic"]
)



Filled 0 missing values with mean
Missing values per feature: 
 Series([], dtype: int64)
Slides available: 436
Omic available: 437
Overlap: 436
Filtering out 1 samples for which there are no omic data available
Dataloader initialised for blca dataset
Dataset: BLCA
Molecular data shape: (436, 2191)
Molecular/Slide match: 436/436
Slide level count: 4
Slide level dimensions: ((79968, 79653), (19992, 19913), (4998, 4978), (2499, 2489))
Slide resize dimensions: w: 1024, h: 1024
Sources selected: ['omic']
Censored share: 0.539
Survival_bin_sizes: {0: 72, 1: 83, 2: 109, 3: 172}
Filled 0 missing values with mean
Missing values per feature: 
 Series([], dtype: int64)
Slides available: 1019
Omic available: 1022
Overlap: 1019
Filtering out 3 samples for which there are no omic data available
Dataloader initialised for brca dataset
Dataset: BRCA
Molecular data shape: (1019, 2922)
Molecular/Slide match: 1019/1019
Slide level count: 3
Slide level dimensions: ((35855, 34985), (8963, 8746), (2240, 218

In [36]:
# get tabular data
blca_loader = DataLoader(
    blca, 
    batch_size=1, 
    shuffle=True, 
    num_workers=multiprocessing.cpu_count()-1
)
[sample], censorship, event_time, y_disc = next(iter(blca_loader))

In [37]:
sample.shape

torch.Size([1, 1, 2183])

## Tabular self-supervised pre-training

To start with, we want to build and encoder-decoder model which trains a cross-attention unit as the encoder, which can later on be deployed in the iterative model. We then want to benchmark the performance with pan-cancer pre-training vs. without pre-training. 

In [38]:
def get_cat_idx(t: torch.Tensor, n_unique: int=20) -> List[int]: 
    """
    Function that takes in a tensor and returns the indices of categorical variables (tensor dimension) 
    A tensor is considered categorical if it has less than n_unique unique values.
    Args:
        t (torch.Tensor): 
        n_unique (int): threshold for unique values

    Returns:
        Tuple(List[int], List[int]): indices of categorical and continuous variables
    """
    cat_idx: List[int] = []
    cont_idx: List[int] = []
    for idx, col in enumerate(t.T): 
        if len(torch.unique(col)) < n_unique:
            cat_idx.append(idx)
        else: 
            cont_idx.append(idx)
    return cat_idx, cont_idx

In [44]:
class AttentionEncoder(nn.Module): 
    """
    Simple encoder that uses fourier encoding, pre-norm and cross-attention to encode the input features into a latent array 
    of size (num_latents x latent_dim). Takes in both the input tensors as well as a randomly initialised latent 
    array as the input. 
    """
    def __init__(self,
                 x_c: int,  # input channels
                 x_d: int,  # channel_dims
                 latent: torch.Tensor,
                 input_axis: int = 1,
                 attn_dropout: float = 0.1,
                 num_heads: int = 4,
                 num_freq_bands: int=8,
                 ):    
        super().__init__()
        
        self.channel_dims = x_d
        self.input_axis = input_axis
        self.attn_dropout = attn_dropout
        self.num_heads = num_heads
        
        
        # fourier_channels = (input_axis * ((num_freq_bands * 2) + 1))
        # input_dim = fourier_channels + input_channels
        input_dim = x_d * x_c
        
        l_c, l_d = latent.shape
        print(l_d, input_dim)
        enc = PreNorm(l_d, Attention(l_d, input_dim, heads=num_heads, dim_head=num_heads, dropout=attn_dropout), context_dim=l_d)
        # enc = Attention(query_dim=input_dim, num_latents=num_latents, latent_dim=latent_dim, heads=num_heads, dim_head=num_heads, dropout=attn_dropout)
        print(f"Input channels {x_c}, latent dim {l_d}")
        
        # attn = LatentCrossAttention(query_dim=input_channels, latent_dim=latent_dim)
        # norm = nn.InstanceNorm1d(latent_dim)
        # enc = PreNorm
        # enc = PreNorm(latent_dim, Attention(query_dim=input_dim, context_dim=latent_dim, heads=num_heads, dim_head=num_heads, dropout=attn_dropout), context_dim=latent_dim)
        self.layers = nn.ModuleList([enc])
        print(self.layers)
        
    def forward(self, x: torch.Tensor, latent: torch.Tensor):
        """
        Note: context is the data, x is the latent
        Args:
            latent: 
            context: 

        Returns:

        """
        for layer in self.layers:
            # note old setup - latent is still the query, context is provided by modality
            latent = layer(query=latent, context=x)
            
        return latent


The decoder often needs to be different depending on the modality, so let's implement modality-specific decoders while trying to have a relatively general-purpose encoder that we can plug into the pipeline.

Note that we may change this later down the line. 

In [40]:
class TabularDecoder(nn.Module):
    """
    Decoder suited for tabular data. We use the following: 
    - Skip connections: faster and more stable training
    - Batch normalisation: stabilises the activations and speeds up training
    - Activation: Output layer to map back to output dimensions, corresponding to the original data dims
    Tries to reconstruct the original input given the latent
    """
    def __init__(self, latent_dim: int, num_latents: int, output_dim: int, method: str = "dense"):
        super(TabularDecoder, self).__init__()
        assert method in ["dense", "conv"], "Decoder type not recognised"
        # check that latent_dim is divisible by 4
        assert num_latents % 4 == 0, "Latent dim must be a multiple of 4"
        layers = []
        
        if method == "dense": 
            
            # flatten latent array (batch, num_latents, latent_dim) -> (batch, num_latents * latent_dim)
            layers.extend([nn.Flatten()]) 
            out_dims = [1024, 512, 256] # may refactor as hyperparameter later
            
            in_dim = latent_dim * num_latents
            for idx, out_dim in enumerate(out_dims):
                
                layers.extend([
                    nn.Linear(in_features=in_dim, out_features=out_dim), 
                    nn.LeakyReLU(), 
                    nn.InstanceNorm1d(out_dim, track_running_stats=False), 
                    nn.Dropout(0.5)
                ])
                
                in_dim = out_dim # update for next layer
            
            # final layer to reconstruct output
            layers.append(nn.Linear(in_dim, output_dim))
        
        elif method == "conv": 
            print(latent_dim, num_latents)
            layers.extend([
                nn.ConvTranspose1d(num_latents, out_channels=int(num_latents/2), kernel_size=4, stride=2, padding=1), 
                nn.BatchNorm1d(int(num_latents/2)),
                nn.LeakyReLU(negative_slope=0.1),
                
                nn.ConvTranspose1d(int(num_latents/2), out_channels=int(num_latents/4), kernel_size=4, stride=2, padding=1),
                nn.BatchNorm1d(int(num_latents/4)),
                nn.LeakyReLU(negative_slope=0.1),
                
                # If you added any other ConvTranspose layers, ensure the channel sizes match correctly for those as well.
                
                nn.Conv1d(int(num_latents/4), out_channels=1, kernel_size=1, stride=1, padding=0)
            ])
        
        self.decode = nn.Sequential(*layers)
        print(self.decode)
        
    def forward(self, latent: torch.Tensor):
        return self.decode(latent)
    
    
        
    

Finally, putting it all together in the encoder-decoder model


In [41]:
from typing import *

class TabPretrainer(nn.Module): 
    """
    Encoder-decoder model for pre-training tabular data.
    # TODO - refactor abstract base class for initialisations 
    """
    def __init__(self,
                 sample: torch.Tensor,
                 latent_shape: List[int],
                 input_axis: int = 1,
                 attn_dropout: float = 0.1,
                 num_heads: int = 4,
                 num_freq_bands: int=8,
                 ):
        super().__init__()
        self.x_c = sample.shape[-2] # number of channels
        self.x_d = sample.shape[-1] # dims per channel (features for tabular data) 
        self.input_axis = input_axis
        self.l_c, self.l_d = latent_shape  # (c x d) [256, 32]
        self.attn_dropout = attn_dropout
        self.num_heads = num_heads
        self.num_freq_bands = num_freq_bands
        
        
        # randomly initialise latent
        self.latent = nn.Parameter(torch.randn(self.l_c, self.l_d))
        
        # encoder
        self.encoder = AttentionEncoder(
            x_c=self.x_c, 
            x_d=self.x_d,
            latent=self.latent,
            input_axis=self.input_axis, 
            attn_dropout=attn_dropout, 
            num_heads=num_heads, 
            num_freq_bands=num_freq_bands
        )
        
        # decoder
        self.decoder = TabularDecoder(
            latent_dim=self.l_d,
            num_latents=self.l_c,
            output_dim=self.x_c,
            method="dense" # using simple encoder to force good representation
        )
        
    def forward(self, x: torch.Tensor):
        
        # expand latent to batch size
        if len(self.latent.shape) == 2:
            b = x.shape[0]
            self.latent = nn.Parameter(einops.repeat(self.latent, "n d -> b n d", b=b))
        
        # encode
        # works much better with skip connections
        self.latent.data = self.encoder(x=x, latent=self.latent).data
        rec_x = self.decoder(self.latent)
        return rec_x
    
    def get_latent(self):
        return self.latent

Next, we need to think about tabular loss functions. Here, we can explore both reconstruction losses and contrastive losses. 

In [42]:
class TabularLoss(nn.Module):
    """
    Reconstruction loss functions for tabular data. We use two types which are commonly used with continuous data: 
    - Mean squared error
    - Constrastive loss, measured as cosine distance between the original and reconstructed data
    We seek to minimise both objectives.
    """
    def __init__(self,
                 method: str = "mse",
                 reduction: str = "mean",
                 ):
        super().__init__()
        assert method in ["mse", "contrastive"], "Loss type not recognised"
        self.loss_type = method
        self.reduction = reduction
        
        if method == "mse":
            self.loss = nn.MSELoss(reduction=reduction)
        elif method == "contrastive":
            self.loss = nn.CosineEmbeddingLoss(reduction=reduction)
            
    def __call__(self, **kwargs):
        return self.loss(**kwargs)
    

Finally, we write a pre-training loop that we can use for pre-training across cancer sites. 

In [43]:
from tqdm import tqdm

torch.set_printoptions(sci_mode=False)


def pretrain_loop(
        data: TCGADataset,
        batch_size: int, 
        epochs: int = 10,
    ):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    loader = DataLoader(
        data, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=multiprocessing.cpu_count()-1
    )
    [omic_sample], _, _, _ = next(iter(loader))
    
    
    model = TabPretrainer(
        sample = omic_sample, 
        input_axis=1, 
        latent_shape=[256, 32], # (l_n x l_d)
        attn_dropout=0.1, 
        num_heads=8,
        num_freq_bands=8
    )
    model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    loss_method = "mse"
    loss_fn = TabularLoss(method=loss_method)
    
    for epoch in tqdm(range(epochs)):
        for idx, batch in enumerate(loader):
            [omic], censorship, event_time, y_disc = batch
            omic = omic.to(device)
            rec_omic = model(omic)
            if loss_method == "contrastive":
                # need to pass in larges for contrastive loss
                # using torch.ones to ensure that omic and rec_omic are learned as similar representations
                # note that this is a slight repurposing of the contrastive loss function
                # with this, the loss is just 1-cos(omic, rec_omic)
                loss = loss_fn(input1=omic, input2=rec_omic, target=torch.ones(omic.shape[0]))
            elif loss_method == "mse": 
                loss = loss_fn(input=omic, target=rec_omic)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            # print every 10th batch
            if idx % 100 == 0:
                pass
                # print(loss)
                # print(omic)
                # print(rec_omic)
        # print epoch-level stats
        print(f"Epoch {epoch+1} loss: {loss}")
        # final reconstruction
        # print error vector
        # print((omic - rec_omic).abs())
        # print(omic)
        # print(rec_omic)
    return model
        
            
    
tab_model = pretrain_loop( data=blca, batch_size=1, epochs=5)
tab_latent = tab_model.get_latent()
    

32 2183
Input channels 1, latent dim 32
ModuleList(
  (0): PreNorm(
    (fn): Attention(
      (to_q): Linear(in_features=32, out_features=64, bias=False)
      (to_kv): Linear(in_features=2183, out_features=128, bias=False)
      (dropout): Dropout(p=0.1, inplace=False)
      (to_out): Sequential(
        (0): Linear(in_features=64, out_features=32, bias=True)
        (1): LeakyReLU(negative_slope=0.01)
      )
    )
    (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (norm_context): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  )
)
Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=8192, out_features=1024, bias=True)
  (2): LeakyReLU(negative_slope=0.01)
  (3): InstanceNorm1d(1024, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (4): Dropout(p=0.5, inplace=False)
  (5): Linear(in_features=1024, out_features=512, bias=True)
  (6): LeakyReLU(negative_slope=0.01)
  (7): InstanceNorm1d(512, eps=1e-05, momentum=0.1, 

  0%|          | 0/5 [00:00<?, ?it/s]


TypeError: forward() missing 1 required positional argument: 'x'

In [ ]:
# get encoder attention weights for a test sample
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = tab_model.encoder.to(device)
encoder.eval()
sample = next(iter(blca_loader))
[omic], _, _, _ = sample
omic = omic.to(device)
# initialise latent
latent = torch.randn(1, 256, 32).to(device)
omic.to(device)
latent.to(device)
# watch out for leakage here
# print(tab_latent)
encoder(query=omic, latent=tab_latent)
encoder.layers[0].attn_weights.shape
encoder.layers[0].attn_weights[0]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_attn_weight_distribution(attn_weights: torch.Tensor):
    attn_weights = attn_weights.cpu().detach().numpy()
    attn_weights = attn_weights.flatten()
    sns.histplot(attn_weights)
    plt.show()
    
def get_most_important_features(attn_weights: torch.Tensor, blca: TCGADataset):
    attn_weights = attn_weights.cpu().detach().numpy()
    attn_weights = attn_weights.flatten()
    # get top 10 features
    top_10 = np.argsort(attn_weights)[-10:]
    # get feature names
    feature_names = blca.omic_df.columns
    top_10_features = feature_names[top_10]
    return top_10_features
    
plot_attn_weight_distribution(encoder.layers[0].attn_weights[0])
get_most_important_features(encoder.layers[0].attn_weights[0], blca)